# Metadata

```{yaml}
Course:   DS 5100
Module:   M05 NumPy
Topic:    NumPy Continued
Author:   R.C. Alvarado (adapted)
Date:     26 June 2022 (revised 13 September 2022)
```

# Quick Refresher on Shape

The **shape** of an array is represented a **tuple**, e.g. `(n, m)` for a 2D array.

* The **length** of the tuple is number of **dimensions** (i.e. axes).
* The **values** of the tuple are the number of **elements** in each dimension (axis).

Consider the array `foo`:

In [1]:
import numpy as np

In [2]:
foo = np.ones((6,4))

In [3]:
foo

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

It was created with the $2$ dimensions.
* The first dimension, axis 0, has $6$ elements. In this case, these elements are arrays.
* The second dimension, axis 1, has $4$ elements. Its elements are scalars (numbers in this case).

The shape of `foo[0]` is $4$. It contains $1$ axis with $4$ elements.

In [4]:
foo[0], foo[0].shape

(array([1., 1., 1., 1.]), (4,))

It has a shape of $1$ and not $4 \times 1$ because it is a vector, not a matrix.

**SO, there is a difference between a vector and a 1-column matrix.**

## Reshaping

If we want to make it into a 1-column matrix, we need to reshape it using `np.reshape()`.

Note that the first value of the shape argument is $-1$. This means use the length of the vector that is passed to it. 

In [5]:
foo2 = np.reshape(foo[0], (-1, 1))

In [6]:
foo2, foo2.shape

(array([[1.],
        [1.],
        [1.],
        [1.]]),
 (4, 1))

When indexing an array, think of the **positions** of the comma-delimitted tuple as the axis.  

The **values** are the **element offsets** in the containing array. The

In [7]:
foo[2:, 2:]
#   ^   ^ 
#   0   1  <- AXIS NUMBER

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])

## Use Case

Sometimes, you need to convert vectors into 1-column matrices to make certain linear algebraic functions work.

Consider two random variables, $x$ and $y$. 

In [127]:
x = np.random.randn(100)
y = np.random.randn(100)

We can fit a regression line using the **normal equation**, which appears in this week's homework.

$\begin{aligned} \hat\beta_i=(x^Tx)^{−1}x^Ty \end{aligned}$ 

Here is a standward way of expressing it in NumPy:

In [10]:
get_beta1 = lambda x, y: (np.linalg.inv(x.T.dot(x))) * (x.T.dot(y))

However, it will fail if we pass it our two variables, `x` and `y`.

The reason is that it expects `x` to be a matrix, since it is designed to handle n-dimension predictor variables, usually represented as $\textbf{X}$. 

In [11]:
get_beta1(x, y)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

The revised function will work with a vector as `x`:

$\hat\beta_i = \large\frac{1}{x^Tx} \small(x^Ty)$

In [22]:
get_beta2 = lambda x, y: (1 / x.T.dot(x)) * (x.T.dot(y))

In [23]:
get_beta2(x, y)

-0.1733770836504587

We can fix the problem in the general case by converting our vector into a matrix using `np.reshape()`.

In [24]:
x_as_matrix = np.reshape(x, (-1, 1))

In [25]:
get_beta1(x_as_matrix, y)

array([[-0.17337708]])

In [26]:
get_beta1(x_as_matrix, y)[0][0]

-0.1733770836504587

In [21]:
get_beta1(x_as_matrix, y)[0][0] == get_beta2(x, y)

True

# Array-Oriented Programming

Using NumPy arrays enables you to express many kinds of data processing tasks as concise array expressions **without writing loops**. 

This practice of **replacing explicit loops with array expressions** is referred to by some people as **vectorization**. 

Vectorized array operations are often significantly **faster** than their pure Python equivalents.

They are also **visually concise and elegant**, although loops have the virtue of visualizing what's under the hood in an algorithm.

# Expressing Conditional Logic as Array Operations

## `np.where()`

The `np.where` function is a vectorized version of the ternary expression `x if condition else y`. 

Suppose we had a boolean array and two arrays of values:

In [38]:
xarr = np.array([1.1, 1.2, 1.3, 1.4, 1.5])
yarr = np.array([2.1, 2.2, 2.3, 2.4, 2.5])
cond = np.array([True, False, True, True, False])

In [39]:
result = [(x if c else y)
          for x, y, c in zip(xarr, yarr, cond)]
result

[1.1, 2.2, 1.3, 1.4, 2.5]

Here is the vectorized version:

In [40]:
result = np.where(cond, xarr, yarr)
result

array([1.1, 2.2, 1.3, 1.4, 2.5])

In [41]:
arr = np.random.randn(4, 4)
arr

array([[-0.23585076, -0.45344237, -0.19564714,  0.07051742],
       [ 0.48147492,  1.85184358,  0.98780609, -0.93677974],
       [ 0.32373982, -0.10250554,  1.38304936, -1.8127358 ],
       [-0.12141417, -0.94532156, -1.35427387,  1.16405348]])

In [42]:
arr > 0
np.where(arr > 0, 2, -2)

array([[-2, -2, -2,  2],
       [ 2,  2,  2, -2],
       [ 2, -2,  2, -2],
       [-2, -2, -2,  2]])

In [43]:
np.where(arr > 0, 2, arr) # set only positive values to 2

array([[-0.23585076, -0.45344237, -0.19564714,  2.        ],
       [ 2.        ,  2.        ,  2.        , -0.93677974],
       [ 2.        , -0.10250554,  2.        , -1.8127358 ],
       [-0.12141417, -0.94532156, -1.35427387,  2.        ]])

# Mathematical and Statistical Methods

Statistical computations are aggregate functions apply to vectors within an array.

In a 2D array, they can be applied to rows or columns, i.e. **axis $0$ or axis $1$**.

Let's create an array of random values. We can think of it is a table of observations and random variables.

In [160]:
arr = np.random.randn(5, 4)

In [166]:
arr

array([[-1.11644156, -0.04974571,  0.14766985, -1.63082928],
       [-0.50545391,  0.06601405,  0.86131383, -0.62499417],
       [ 0.58951036, -0.62322748, -0.49359291, -0.79002809],
       [ 0.09873484, -0.0929257 ,  0.59170603, -0.81527978],
       [-0.86287813,  0.33746569,  0.25063287, -0.53800917]])

## `.mean()`

In [169]:
arr.mean()

-0.26001791821779613

In [170]:
np.mean(arr)

-0.26001791821779613

In [171]:
arr.mean(axis=1)

array([-0.66233668, -0.05078005, -0.32933453, -0.05444115, -0.20319719])

## `.sum()`

In [175]:
arr.sum()

-5.200358364355923

Row wise aggregration

In [176]:
arr.sum(axis=0)

array([-1.7965284 , -0.36241914,  1.35772966, -4.39914049])

In [177]:
arr.mean(axis=1)

array([-0.66233668, -0.05078005, -0.32933453, -0.05444115, -0.20319719])

Column-wise aggregration

## `.cumsum()`

In [178]:
arr = np.array([0, 1, 2, 3, 4, 5, 6, 7])
arr.cumsum()

array([ 0,  1,  3,  6, 10, 15, 21, 28])

In [179]:
arr = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [180]:
arr.cumsum(axis=0)

array([[ 0,  1,  2],
       [ 3,  5,  7],
       [ 9, 12, 15]])

In [181]:
arr.cumprod(axis=1)

array([[  0,   0,   0],
       [  3,  12,  60],
       [  6,  42, 336]])

# Methods for Boolean Arrays

## `.sum()`

Since booleans are $0$s and $1$, we can sum them to get a total truth count.

In [182]:
arr = np.random.randn(100)
(arr > 0).sum() # Number of positive values

45

In [183]:
bools = np.array([False, False, True, False])

## `.any()`

In [184]:
bools.any()

True

## `.all()`

In [185]:
bools.all()

False

# Sorting 

## `.sort()`

In [186]:
arr = np.random.randn(6)
arr

array([-0.74980556, -0.45216793,  0.95432456,  0.50730782, -0.69393824,
       -0.47149034])

In [187]:
arr.sort()
arr

array([-0.74980556, -0.69393824, -0.47149034, -0.45216793,  0.50730782,
        0.95432456])

In [188]:
arr = np.random.randn(5, 3)
arr

array([[ 0.52978719,  1.26831824, -0.08892941],
       [ 0.07856138, -0.03757781,  0.61640601],
       [ 0.34139245,  0.59608407, -0.56890453],
       [-0.89128198, -0.05968455,  1.0120217 ],
       [-1.29346258, -0.04988416, -1.69407724]])

In [189]:
arr.sort(1)
arr

array([[-0.08892941,  0.52978719,  1.26831824],
       [-0.03757781,  0.07856138,  0.61640601],
       [-0.56890453,  0.34139245,  0.59608407],
       [-0.89128198, -0.05968455,  1.0120217 ],
       [-1.69407724, -1.29346258, -0.04988416]])

In [190]:
large_arr = np.random.randn(1000)
large_arr.sort()
large_arr[int(0.05 * len(large_arr))] # 5% quantile

-1.5931479349694544

In [191]:
0.05 * len(large_arr)

50.0

# Unique and Other Set Logic

## `np.unique()`

In [192]:
names = np.array(['Bob', 'Joe', 'Will', 'Bob', 'Will', 'Joe', 'Joe'])
np.unique(names)

array(['Bob', 'Joe', 'Will'], dtype='<U4')

In [193]:
sorted(set(names))

['Bob', 'Joe', 'Will']

In [194]:
ints = np.array([3, 3, 3, 2, 2, 1, 1, 4, 4])
np.unique(ints)

array([1, 2, 3, 4])

## `np.in1d()`

Tests whether each element of a 1-D array is also present in a second array.

In [195]:
values = np.array([6, 0, 0, 3, 2, 5, 6])
np.in1d(values, [2, 3, 6])

array([ True, False, False,  True,  True, False,  True])

# File Input and Output with Arrays

## `np.save()`

Save an array to a binary file in NumPy ``.npy`` format.

Automatically adds the `.npy` file extension.

In [196]:
arr = np.arange(10)
np.save('some_array', arr)

## `np.load()`

In [197]:
np.load('some_array.npy')

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## `np.savetxt()`

Save an array to a text file.

In [198]:
# np.savetxt?

In [199]:
np.savetxt('some_array.txt', arr)

In [ ]:
!more some_array.txt

0.000000000000000000e+00
1.000000000000000000e+00
2.000000000000000000e+00
3.000000000000000000e+00
4.000000000000000000e+00
5.000000000000000000e+00
6.000000000000000000e+00
7.000000000000000000e+00
8.000000000000000000e+00
9.000000000000000000e+00
some_array.txt (END)

## `np.savez()`

Save several arrays into a single file in uncompressed ``.npz`` format.

In [ ]:
np.savez('array_archive.npz', a=arr, b=arr)

In [ ]:
arch = np.load('array_archive.npz')
arch['b']

In [ ]:
np.savez_compressed('arrays_compressed.npz', a=arr, b=arr)

Clean up ...

In [ ]:
!rm some_array.npy
!rm some_array.txt
!rm array_archive.npz
!rm arrays_compressed.npz

# Linear Algebra

## `.dot()`

Dot product of two arrays. Specifically,
- If both `a` and `b` are 1-D arrays, it is inner product of vectors
  (without complex conjugation).
- If both `a` and `b` are 2-D arrays, it is matrix multiplication,
  but using `matmul()` or `a @ b` is preferred.
- If either `a` or `b` is 0-D (scalar), it is equivalent to `multiply()`
  and using ``numpy.multiply(a, b)`` or ``a * b`` is preferred.
- If `a` is an N-D array and `b` is a 1-D array, it is a sum product over
  the last axis of `a` and `b`.
- If `a` is an N-D array and `b` is an M-D array (where `M>=2`), it is a
  sum product over the last axis of `a` and the second-to-last axis of `b`:
  
```
    dot(a, b)[i,j,k,m] = sum(a[i,j,:] * b[k,:,m])
```

In [ ]:
x = np.array([[1., 2., 3.], [4., 5., 6.]])
x

In [ ]:
y = np.array([[6., 23.], [-1, 7], [8, 9]])
y

In [ ]:
x.dot(y)

In [ ]:
np.dot(x, y)

In [ ]:
np.dot(x, np.ones(3))

In NumPy, the `@` operator means [matrix multiplication](https://www.codingem.com/numpy-at-operator/).

In [ ]:
x @ np.ones(3)

## `np.linalg.inv()`

In [ ]:
from numpy.linalg import inv, qr
X = np.random.randn(5, 5)
mat = X.T.dot(X)
inv(mat)

In [ ]:
mat.dot(inv(mat))
q, r = qr(mat)
r

# Pseudorandom Number Generation

## `np.random.normal()`

In [ ]:
samples = np.random.normal(size=(4, 4))
samples

In [ ]:
from random import normalvariate
N = 1000000
%timeit samples = [normalvariate(0, 1) for _ in range(N)]
%timeit np.random.normal(size=N)

In [ ]:
np.random.seed(1234)

In [ ]:
rng = np.random.RandomState(1234)

In [ ]:
rng.randn(10)

## Example: Random Walks

Let simulate a random walk. The walk will be represented as a vector.

We'll do it first as loop, then with vectorization.

**Loops**

In [ ]:
import random
position = 0
walk = [position] # initialize the walk
steps = 1000
for i in range(steps):
    step = 1 if random.randint(0, 1) else -1  # Coin toss
    position += step
    walk.append(position)

In [ ]:
plt.figure()
plt.plot(walk[:100]);

**Vectors**

In [ ]:
np.random.seed(12345)

In [ ]:
nsteps = 1000
draws = np.random.randint(0, 2, size=nsteps)
steps = np.where(draws > 0, 1, -1)
walk2 = steps.cumsum()

In [ ]:
plt.plot(walk[:100]);

In [ ]:
walk2.min(), walk2.max()

## Simulating Many Random Walks at Once

In [ ]:
nwalks = 5000
nsteps = 1000

In [ ]:
draws = np.random.randint(0, 2, size=(nwalks, nsteps)) # 0 or 1
steps = np.where(draws > 0, 1, -1)
walks = steps.cumsum(1)

In [ ]:
walks

In [ ]:
walks.max(), walks.min()

In [ ]:
hits30 = (np.abs(walks) >= 30).any(1)
hits30

In [ ]:
hits30.sum() # Number that hit 30 or -30

In [ ]:
crossing_times = (np.abs(walks[hits30]) >= 30).argmax(1)
crossing_times.mean()

In [ ]:
steps = np.random.normal(loc=0, scale=0.25, size=(nwalks, nsteps))

Feel free to experiment with other distributions for the steps other than equal-sized coin flips. You need only use a different random generator method, like standard_normal to generate normally distributed steps with some mean and standard deviation:

In [ ]:
draws = 0.25 * rng.standard_normal((nwalks, nsteps)) # Walks: observations, Steps: features

In [ ]:
sample_walk = np.random.choice(len(draws))
plt.plot(draws[sample_walk])
plt.title(f"Walk #{sample_walk}");